In [1]:
import numpy as np
import pandas as pd
#import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
import collections
import itertools as it
from sklearn import ensemble, linear_model
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
val = pd.read_csv('validation.csv')
test = pd.read_csv('test.csv')

In [7]:
# Tune base_bid on VAL SET

pCTR_val = np.loadtxt('CTR_models/gbm_dam/pCTR_gbm_dam_val.csv')
CTR_frac = pCTR_val / np.mean(pCTR_val)
print(CTR_frac.shape)

start = time.time()
clicks = []
budget = 6250 * 1000
for base_bid in range(30,150):
    
    bid = base_bid * CTR_frac 
    df = val[val['payprice'] < bid][['click','payprice']]
    
    cost = df['payprice'].cumsum()
    eligible = df[cost < budget]
    
    #print([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])
    clicks.append([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])

print(time.time()-start)    
clicks  

(303925,)
6.3973658084869385


[[30, 104, 989485],
 [31, 104, 1041048],
 [32, 105, 1096313],
 [33, 106, 1145595],
 [34, 107, 1203576],
 [35, 107, 1261597],
 [36, 107, 1322996],
 [37, 107, 1381107],
 [38, 107, 1438786],
 [39, 108, 1496980],
 [40, 109, 1559630],
 [41, 112, 1621456],
 [42, 113, 1686069],
 [43, 113, 1749344],
 [44, 114, 1809053],
 [45, 115, 1876996],
 [46, 115, 1941160],
 [47, 116, 2011072],
 [48, 117, 2073760],
 [49, 119, 2136823],
 [50, 119, 2199812],
 [51, 119, 2269769],
 [52, 120, 2332514],
 [53, 120, 2401221],
 [54, 121, 2467046],
 [55, 122, 2530970],
 [56, 122, 2590944],
 [57, 124, 2658102],
 [58, 125, 2719332],
 [59, 125, 2779928],
 [60, 125, 2840786],
 [61, 127, 2912069],
 [62, 128, 2978635],
 [63, 128, 3046934],
 [64, 129, 3110972],
 [65, 129, 3175789],
 [66, 129, 3239996],
 [67, 129, 3304194],
 [68, 129, 3374856],
 [69, 129, 3443392],
 [70, 130, 3513092],
 [71, 130, 3584052],
 [72, 131, 3650902],
 [73, 132, 3723108],
 [74, 132, 3792366],
 [75, 133, 3864117],
 [76, 134, 3926539],
 [77, 134, 399

In [6]:
# Calculate bids for submission

pCTR_test = np.loadtxt('CTR_models/gbm_dam/pCTR_gbm_dam_test.csv')

CTR_frac = pCTR_test / np.mean(pCTR_test)
base_bid = 113 # 151 clicks on val
final_bids = np.round(base_bid * CTR_frac * np.linspace(0.9,1.1,len(pCTR_test)))

submission = np.array([test['bidid'], final_bids])
print(submission[:10])

tmp = pd.DataFrame(submission.T, columns=['bidid','bidprice'])
tmp.head()
print("head", tmp.head())
print("tail", tmp.tail())
print("mean bid", np.mean(final_bids))


[['366c563de7d90feb9d4dab53e795a93fb3157387'
  '29167d4caa719788b5a342dbaa25151d53121f80'
  'ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e' ...
  'edb83d1e92bab4113360a655b68c2e44ace6a81d'
  '5eca186cb78a6eaadf91aa7070c88f2c25e31464'
  'afcdff59c976336fbcecb98b2c7379d369496131']
 [77.0 312.0 12.0 ... 11.0 153.0 27.0]]
head                                       bidid bidprice
0  366c563de7d90feb9d4dab53e795a93fb3157387       77
1  29167d4caa719788b5a342dbaa25151d53121f80      312
2  ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e       12
3  844c2da00d45315f20b748ec131c26ee99a7cbc7       49
4  c6017f0ad0c44d7d0c9b62583ea863f28941c0ca      108
tail                                            bidid bidprice
303370  c3bb9e4340efd0b7727e1340be7975856878e27c       40
303371  75a0dac04603fa79e811f1c3e7d1a309f5a81bbf       11
303372  edb83d1e92bab4113360a655b68c2e44ace6a81d       11
303373  5eca186cb78a6eaadf91aa7070c88f2c25e31464      153
303374  afcdff59c976336fbcecb98b2c7379d369496131       27
mean bid 